### At this stage, we will start creating library called MiniFLow, which is like our own version of Tensorflow .. 

Class Architecture:

--  **Neuron** [ Base Class ] <br>
----  **Input** [ Sub-Class which is only used as Input Layer Node, i.e. performing no operation ]<br>
----  **Add** [ Sub-Class which is only used as Hidden Layer Node. i.e. performing some operation ]<br>

Each of above class has method name foward like Input.forward() or Add.foward() -- these are basically computing values which these are expected to foward, in case of Input it is simply placeholder.



In [36]:
"""
You need to change the Add() class below.
"""

class Neuron:
    def __init__(self, inbound_neurons=[]):
        # Neurons from which this Node receives values
        self.inbound_neurons = inbound_neurons
        # Neurons to which this Node passes values
        self.outbound_neurons = []
        # A calculated value
        self.value = None
        # Add this node as an outbound node on its inputs.
        for n in self.inbound_neurons:
            print ("In outbound loop", n)
            n.outbound_neurons.append(self)
            print ("Outbound Neurons", len(n.outbound_neurons))
            
        print ("Inbound Neurons", len(self.inbound_neurons))
        

    # These will be implemented in a subclass.
    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_neurons` and
        store the result in self.value.
        """
        raise NotImplemented


class Input(Neuron):
    def __init__(self):
        # an Input neuron has no inbound nodes,
        # so no need to pass anything to the Node instantiator
        Neuron.__init__(self)

    # NOTE: Input node is the only node where the value
    # is passed as an argument to forward().
    #
    # All other neuron implementations should get the value
    # of the previous neurons from self.inbound_neurons
    #
    # Example:
    # val0 = self.inbound_neurons[0].value
    def forward(self, value=None):
        # Overwrite the value if one is passed in.
        if value is not None:
            self.value = value
        print ("In Forward of Input...")


class Add(Neuron):
    def __init__(self, x, y):
        # You could access `x` and `y` in forward with
        # self.inbound_neurons[0] (`x`) and self.inbound_neurons[1] (`y`)
        Neuron.__init__(self, [x, y])
        #print("xval, yval", x.value, y.value)

    def forward(self):
        """
        Set the value of this neuron (`self.value`) to the sum of it's inbound_nodes.
        
        Your code here!
        """
        x_val= self.inbound_neurons[0].value
        y_val= self.inbound_neurons[1].value
        self.value = x_val+y_val
        #print("xval, yval", x_val, y_val)
        print ("In Forward of ADD...")

"""
No need to change anything below here!
"""

'\nNo need to change anything below here!\n'

##### Helper Functions

In [37]:
## Sort generic nodes in topological order using Kahn's Algorithm.
def topological_sort(feed_dict):
    """
    Sort generic nodes in topological order using Kahn's Algorithm.

    `feed_dict`: A dictionary where the key is a `Input` node and the value is the respective value feed to that node.

    Returns a list of sorted nodes.
    """

    input_neurons = [n for n in feed_dict.keys()]
    
    G = {}
    neurons = [n for n in input_neurons]
    
    while len(neurons) > 0:
        n = neurons.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.outbound_neurons:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            neurons.append(m)

    L = []
    S = set(input_neurons)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.value = feed_dict[n]

        L.append(n)
        for m in n.outbound_neurons:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            # if no other incoming edges add to S
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


##Performs a forward pass through a list of sorted neurons.
def forward_pass(output_neuron, sorted_neurons):
    """
    Performs a forward pass through a list of sorted neurons.

    Arguments:

        `output_neuron`: A neuron in the graph, should be the output neuron (have no outgoing edges).
        `sorted_neurons`: a topologically sorted list of neurons.

    Returns the output neuron's value
    """

    for n in sorted_neurons:
        n.forward()

    return output_neuron.value

### Let's Implement Neural Nerowrk using above defined library called Miniflow .. 

In [38]:
"""
This script builds and runs a graph with miniflow.

There is no need to change anything to solve this quiz!

However, feel free to play with the network! Can you also
build a network that solves the equation below?

(x + y) + y
"""
# use this import when defined as py files
#from miniflow import *

print ("Inbound/Outbound at X & Y")
x, y = Input(), Input()
print("")

print ("Inbound/Outbound at ADD")
f = Add(x, y)
print("")

feed_dict = {x: 10, y: 5}

sorted_neurons = topological_sort(feed_dict)
output = forward_pass(f, sorted_neurons)

# NOTE: because topological_sort set the values for the `Input` neurons we could also access
# the value for x with x.value (same goes for y).
print("{} + {} = {} (according to miniflow)".format(feed_dict[x], feed_dict[y], output))

Inbound/Outbound at X & Y
('Inbound Neurons', 0)
('Inbound Neurons', 0)

Inbound/Outbound at ADD
('In outbound loop', <__main__.Input instance at 0x1061e0a28>)
('Outbound Neurons', 1)
('In outbound loop', <__main__.Input instance at 0x106087c20>)
('Outbound Neurons', 1)
('Inbound Neurons', 2)

In Forward of Input...
In Forward of Input...
In Forward of ADD...
10 + 5 = 15 (according to miniflow)
